# Neural Networks: Logistic Regression

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters
INPUT_SIZE = 28 * 28
NUM_CLASSES = 10
NUM_EPOCHS = 5
BATCH_SIZE = 100
LEARNING_RATE = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transforms.ToTensor()
)

In [4]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

### Logistic Regression

We will use the code below to implement a simple logistic regression model with PyTorch.


In [5]:
# Logistic regression model
model = nn.Linear(INPUT_SIZE, NUM_CLASSES)

# loss and optimizer
criterion = nn.CrossEntropyLoss()  # computes softmax and then the cross entropy
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [6]:
# Train the model
total_step = len(train_loader)

for epoch in range(NUM_EPOCHS):
    # Batch training - load a batch of images with its (index, data, class)
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        # - Compress images to flatten them into a single vector
        images = images.reshape(-1, INPUT_SIZE)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, NUM_EPOCHS, i+1, total_step, loss.item()))
    print()

Epoch [1/5], Step [100/600], Loss: 2.1954
Epoch [1/5], Step [200/600], Loss: 2.0851
Epoch [1/5], Step [300/600], Loss: 2.0077
Epoch [1/5], Step [400/600], Loss: 1.9011
Epoch [1/5], Step [500/600], Loss: 1.8679
Epoch [1/5], Step [600/600], Loss: 1.7904

Epoch [2/5], Step [100/600], Loss: 1.7167
Epoch [2/5], Step [200/600], Loss: 1.6531
Epoch [2/5], Step [300/600], Loss: 1.6013
Epoch [2/5], Step [400/600], Loss: 1.6070
Epoch [2/5], Step [500/600], Loss: 1.5111
Epoch [2/5], Step [600/600], Loss: 1.5394

Epoch [3/5], Step [100/600], Loss: 1.4825
Epoch [3/5], Step [200/600], Loss: 1.4080
Epoch [3/5], Step [300/600], Loss: 1.2766
Epoch [3/5], Step [400/600], Loss: 1.3056
Epoch [3/5], Step [500/600], Loss: 1.2613
Epoch [3/5], Step [600/600], Loss: 1.2310

Epoch [4/5], Step [100/600], Loss: 1.2142
Epoch [4/5], Step [200/600], Loss: 1.2237
Epoch [4/5], Step [300/600], Loss: 1.1361
Epoch [4/5], Step [400/600], Loss: 1.1614
Epoch [4/5], Step [500/600], Loss: 1.1392
Epoch [4/5], Step [600/600], Lo

In [7]:
outputs.data.shape

torch.Size([100, 10])

In [8]:
torch.max(outputs.data, 1).values.shape

torch.Size([100])

In [9]:
# Test the model
# In the test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, INPUT_SIZE)
        
        # Forward pass
        outputs = model(images)
        
        # Get predictions
        _, predicted = torch.max(outputs.data, 1)
        
        # Total number of labels
        total += labels.size(0)
        
        # Total correct predictions
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 1000 test images: {:.2f} %'.format(100 * correct / total))

Accuracy of the model on the 1000 test images: 82.98 %


In [10]:
# Save the model checkpoint
torch.save(model.state_dict(), 'lr-model.ckpt')